In [ ]:
%reset -f
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
plt.close('all')

import numpy as np

import sys
sys.path.append('../')

import pcmls

import jax
jax.config.update('jax_enable_x64', True)
jax.config.update('jax_debug_nans', True)

## Generate Point Set for Evaluation

In [ ]:
# Random points defined over [0 1] X [0 1]
# num_points = 2500
# # np.random.seed(42) # For reproducible random numbers
# P = random.uniform(0, 1, num_points)

# A uniform grid defined over [0 1] X [0 1]
px, py = np.meshgrid(np.linspace(0, 1, 40), np.linspace(0, 1, 40))
P = np.column_stack((px.ravel(), py.ravel()))
del px, py

U = pcmls.franke(P[:,0], P[:,1])
grad_U = pcmls.grad_franke(P[:,0], P[:,1])
hess_U = pcmls.hess_franke(P[:,0], P[:,1])

## Evaluate Franke's Function and Derivatives

In [ ]:
Fq, DFq, HFq = pcmls.pcmls(P, P, U, m=4, weight_function='wendland',
                     weight_method='knn', weight_param=30, vectorized=False,
                     compute_gradients=True, compute_hessians=True,
                     verbose=True)

In [ ]:
plt.close('all')

func_err = np.abs(Fq - U) / np.abs(U)

print('Max function error = ', np.max(func_err))
print('Mean function error = ', np.mean(func_err))
print('Median function error = ', np.median(func_err))

fig, axs = plt.subplots(1, 3, subplot_kw={'projection': '3d'}, figsize=(15, 6))

sc1 = axs[0].scatter(P[:,0], P[:,1], U, c=U, cmap='viridis')
axs[0].set_title('True Function')
axs[0].set_xlabel('X')
axs[0].set_ylabel('Y')
axs[0].set_zlabel('Z')
axs[0].view_init(elev=90, azim=-90)
axs[0].set_proj_type('ortho')
sc1.set_clim(np.min(U), np.max(U))
fig.colorbar(sc1, ax=axs[0], label='U')

sc2 = axs[1].scatter(P[:,0], P[:,1], Fq, c=Fq, cmap='viridis')
axs[1].set_title('MLS Function')
axs[1].set_xlabel('X')
axs[1].set_ylabel('Y')
axs[1].set_zlabel('Z')
axs[1].view_init(elev=90, azim=-90)
axs[1].set_proj_type('ortho')
sc2.set_clim(np.min(U), np.max(U))
fig.colorbar(sc2, ax=axs[1], label='U')

sc3 = axs[2].scatter(P[:,0], P[:,1], func_err, c=func_err, cmap='viridis')
axs[2].set_title('Relative Error')
axs[2].set_xlabel('X')
axs[2].set_ylabel('Y')
axs[2].set_zlabel('Z')
axs[2].view_init(elev=90, azim=-90)
axs[2].set_proj_type('ortho')
sc3.set_clim(0, 0.1)
fig.colorbar(sc3, ax=axs[2], label='Rel Error')

plt.tight_layout()
plt.show()

In [ ]:
plt.close('all')

true_grad_norm = np.sqrt(np.sum(grad_U**2, axis=1))
mls_grad_norm = np.sqrt(np.sum(DFq**2, axis=1))
grad_err = np.sqrt(np.sum((grad_U-DFq)**2, axis=1)) / true_grad_norm

print('Max gradient error =', np.max(grad_err))
print('Mean gradient error = ', np.mean(grad_err))
print('Median gradient error = ', np.median(grad_err))

fig, axs = plt.subplots(1, 3, figsize=(15, 6))

sc1 = axs[0].scatter(P[:,0], P[:,1], c=true_grad_norm, cmap='viridis')
quiv1 = axs[0].quiver(P[:,0], P[:,1], grad_U[:,0], grad_U[:,1],
                      color='m', width=3e-4*15)
axs[0].set_title('True Function Gradient')
axs[0].set_xlabel('X')
axs[0].set_ylabel('Y')
axs[0].set_aspect('equal')
sc1.set_clim(np.min(true_grad_norm), np.max(true_grad_norm))
fig.colorbar(sc1, ax=axs[0], label=r'$\nabla U$')

sc2 = axs[1].scatter(P[:,0], P[:,1], c=mls_grad_norm, cmap='viridis')
quiv2 = axs[1].quiver(P[:,0], P[:,1], DFq[:,0], DFq[:,1],
                      color='m', width=3e-4*15)
axs[1].set_title('MLS Function Gradient')
axs[1].set_xlabel('X')
axs[1].set_ylabel('Y')
axs[1].set_aspect('equal')
sc2.set_clim(np.min(true_grad_norm), np.max(true_grad_norm))
fig.colorbar(sc2, ax=axs[1], label=r'$\nabla U$')

sc3 = axs[2].scatter(P[:,0], P[:,1], c=grad_err, cmap='viridis')
axs[2].set_title('Relative Gradient Error')
axs[2].set_xlabel('X')
axs[2].set_ylabel('Y')
axs[2].set_aspect('equal')
sc3.set_clim(0, 0.1)
fig.colorbar(sc3, ax=axs[2], label='Rel Error')

plt.tight_layout()
plt.show()

In [ ]:
plt.close('all')

true_hess_norm = np.sqrt(np.sum(np.sum(hess_U**2, axis=2), axis=1))
hess_err = np.sqrt(np.sum(np.sum((hess_U-HFq)**2, axis=2), axis=1))  / true_hess_norm
hess_err_xx = np.sqrt((hess_U[:,0,0]-HFq[:,0,0])**2) / np.sqrt(hess_U[:,0,0]**2)
hess_err_xy = np.sqrt((hess_U[:,0,1]-HFq[:,0,1])**2) / np.sqrt(hess_U[:,0,1]**2)
hess_err_yy = np.sqrt((hess_U[:,1,1]-HFq[:,1,1])**2) / np.sqrt(hess_U[:,1,1]**2)

print('Max Hessian error =', np.max(hess_err))
print('Mean Hessian error = ', np.mean(hess_err))
print('Median Hessian error = ', np.median(hess_err))

fig, axs = plt.subplots(3, 3, figsize=(15, 15))

sc11 = axs[0,0].scatter(P[:,0], P[:,1], c=hess_U[:,0,0], cmap='viridis')
axs[0,0].set_title('True Hxx')
axs[0,0].set_xlabel('X')
axs[0,0].set_ylabel('Y')
axs[0,0].set_aspect('equal')
sc11.set_clim(np.min(hess_U[:,0,0]), np.max(hess_U[:,0,0]))
fig.colorbar(sc11, ax=axs[0,0])

sc12 = axs[0,1].scatter(P[:,0], P[:,1], c=hess_U[:,0,1], cmap='viridis')
axs[0,1].set_title('True Hxy')
axs[0,1].set_xlabel('X')
axs[0,1].set_ylabel('Y')
axs[0,1].set_aspect('equal')
sc12.set_clim(np.min(hess_U[:,0,1]), np.max(hess_U[:,0,1]))
fig.colorbar(sc12, ax=axs[0,1])

sc13 = axs[0,2].scatter(P[:,0], P[:,1], c=hess_U[:,1,1], cmap='viridis')
axs[0,2].set_title('True Hyy')
axs[0,2].set_xlabel('X')
axs[0,2].set_ylabel('Y')
axs[0,2].set_aspect('equal')
sc13.set_clim(np.min(hess_U[:,1,1]), np.max(hess_U[:,1,1]))
fig.colorbar(sc13, ax=axs[0,2], label='Hyy')

sc21 = axs[1,0].scatter(P[:,0], P[:,1], c=HFq[:,0,0], cmap='viridis')
axs[1,0].set_title('MLS Hxx')
axs[1,0].set_xlabel('X')
axs[1,0].set_ylabel('Y')
axs[1,0].set_aspect('equal')
sc21.set_clim(np.min(hess_U[:,0,0]), np.max(hess_U[:,0,0]))
fig.colorbar(sc21, ax=axs[1,0])

sc22 = axs[1,1].scatter(P[:,0], P[:,1], c=HFq[:,0,1], cmap='viridis')
axs[1,1].set_title('MLS Hxy')
axs[1,1].set_xlabel('X')
axs[1,1].set_ylabel('Y')
axs[1,1].set_aspect('equal')
sc22.set_clim(np.min(hess_U[:,0,1]), np.max(hess_U[:,0,1]))
fig.colorbar(sc22, ax=axs[1,1])

sc23 = axs[1,2].scatter(P[:,0], P[:,1], c=HFq[:,1,1], cmap='viridis')
axs[1,2].set_title('MLS Hyy')
axs[1,2].set_xlabel('X')
axs[1,2].set_ylabel('Y')
axs[1,2].set_aspect('equal')
sc23.set_clim(np.min(hess_U[:,1,1]), np.max(hess_U[:,1,1]))
fig.colorbar(sc23, ax=axs[1,2])

sc31 = axs[2,0].scatter(P[:,0], P[:,1], c=hess_err_xx, cmap='viridis')
axs[2,0].set_title('Relative Hxx Error')
axs[2,0].set_xlabel('X')
axs[2,0].set_ylabel('Y')
axs[2,0].set_aspect('equal')
sc31.set_clim(0, 0.1)
fig.colorbar(sc31, ax=axs[2,0])

sc32 = axs[2,1].scatter(P[:,0], P[:,1], c=hess_err_xy, cmap='viridis')
axs[2,1].set_title('Relative Hxy Error')
axs[2,1].set_xlabel('X')
axs[2,1].set_ylabel('Y')
axs[2,1].set_aspect('equal')
sc32.set_clim(0, 0.1)
fig.colorbar(sc32, ax=axs[2,1])

sc33 = axs[2,2].scatter(P[:,0], P[:,1], c=hess_err_yy, cmap='viridis')
axs[2,2].set_title('Relative Hyy Error')
axs[2,2].set_xlabel('X')
axs[2,2].set_ylabel('Y')
axs[2,2].set_aspect('equal')
sc33.set_clim(0, 0.1)
fig.colorbar(sc33, ax=axs[2,2])